## 데이터 불러오기

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re    
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수:', len(lines))
lines.sample(5)

전체 샘플의 수: 217975


,eng,fra,cc
146630,This can't be what it looks like.,Ça ne peut pas être ce que ça paraît.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
70716,I just got back in town.,Je viens de revenir en ville.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
145702,Please don't make me go with Tom.,"S'il te plaît, ne m'oblige pas à y aller avec ...",CC-BY 2.0 (France) Attribution: tatoeba.org #3...
40855,It's a little dated.,C'est un peu désuet.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
24483,Aren't you sleepy?,N'as-tu pas sommeil ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [3]:
lines = lines[['eng', 'fra']][60000:93000]
lines.sample(5)

,eng,fra
89192,I'm disappointed with you.,Tu me déçois.
69825,His house is by a river.,Sa maison est au bord de la rivière.
69576,He is quite a gentleman.,C'est un gentleman.
68971,Do you like this garden?,Vous aimez ce jardin ?
86345,He lived there by himself.,Il vivait là seul.


In [4]:
lines_np_eng= lines['eng'].to_numpy()
lines_np_fra= lines['fra'].to_numpy()
lines_np_eng

array(['You were there, right?', 'You will stay at home.',
       "You won't be punished.", ..., 'What time did you wake up?',
       'What time did you wake up?', 'What time did you wake up?'],
      dtype=object)

## 정제, 정규화, 전처리 하기

### 소문자 변경 후 구두점 분리 함수

In [5]:
sos_token = '<start> '
eos_token = ' <end>'

def preprocess_line(line, plus_token = True):
    # 소문자로 변경하기
    line = line.lower().strip()
    # 구두점(Punctuation)을 단어와 분리하기
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line)

    line = line.strip()
    
    if plus_token == True:
        line = sos_token + line + eos_token
    
    return line

### 띄어쓰기 단위로 토큰화 함수

In [6]:
def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=7000,  
        filters=' ',   
        oov_token="<unk>"  
    )
    tokenizer.fit_on_texts(corpus)  

    tensor = tokenizer.texts_to_sequences(corpus)   

    return tensor, tokenizer

### 영어, 프랑스어 전처리하기

In [7]:
eng_lines = []
fra_lines = []

# eng_lines.append(lines.eng.apply(lambda x : preprocess_line(x,plus_token = False)))
# fra_lines.append(lines.fra.apply(lambda x : preprocess_line(x),))

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    eng_lines.append(preprocess_line(eng, plus_token = False))
    fra_lines.append(preprocess_line(fra))

In [8]:
np.shape(eng_lines)

(33000,)

In [9]:
eng_tensor, eng_tokenizer = tokenize(eng_lines)
fra_tensor, fra_tokenizer = tokenize(fra_lines)
fra_tensor[:10]

[[2, 15, 105, 23, 374, 34, 19, 7, 22, 8, 6, 3],
 [2, 15, 3036, 151, 78, 4, 3],
 [2, 15, 13, 1135, 8, 3723, 4, 3],
 [2, 10, 19, 347, 8, 16, 1519, 4, 3],
 [2, 15, 13, 389, 8, 16, 1519, 4, 3],
 [2, 10, 19, 1647, 8, 17, 4, 3],
 [2, 15, 443, 176, 9, 5098, 4, 3],
 [2, 15, 443, 176, 9, 35, 20, 128, 278, 4, 3],
 [2, 15, 443, 176, 9, 26, 841, 72, 4, 3],
 [2, 10, 1136, 176, 9, 10, 841, 72, 4, 3]]

### input, target 설정

In [10]:
encoder_input = eng_tensor
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index['<end>']] for line in fra_tensor]
# 시작 토큰 제거
decoder_target =[[char for char in line if char != fra_tokenizer.word_index['<start>']] for line in fra_tensor]

### padding 추가

In [11]:
def pad_tensor(tensor):
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = max(num_tokens)
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    tensor = pad_sequences(tensor, padding='post', maxlen=maxlen)  
    return tensor

In [12]:
encoder_input = pad_tensor(encoder_input)
decoder_input = pad_tensor(decoder_input)
decoder_target = pad_tensor(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [13]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1

max_eng_seq_len = encoder_input.shape[1]
max_fra_seq_len = decoder_input.shape[1]

In [14]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 5795
프랑스어 단어장의 크기 : 8297
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 17


## train, test dataset 나누기

In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 10)
(30000, 17)
(30000, 17)
(3000, 10)
(3000, 17)
(3000, 17)


## 임베딩 층(Embedding layer) 사용하기

### 인코더 

In [17]:
embedding_size = 512
hidden_size = 512
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

### 디코더

In [18]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [19]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 512)    2967040     encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 512)    4248064     decoder_input[0][0]              
______________________________________________________________________________________________

In [22]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=50)

Epoch 1/50
938/938 [==============================] - 54s 21ms/step - loss: 1.9346 - val_loss: 1.5865
Epoch 2/50
938/938 [==============================] - 17s 18ms/step - loss: 1.4684 - val_loss: 1.3613
Epoch 3/50
938/938 [==============================] - 17s 18ms/step - loss: 1.2802 - val_loss: 1.2470
Epoch 4/50
938/938 [==============================] - 17s 18ms/step - loss: 1.1495 - val_loss: 1.1510
Epoch 5/50
938/938 [==============================] - 17s 19ms/step - loss: 1.0476 - val_loss: 1.0886
Epoch 6/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9663 - val_loss: 1.0442
Epoch 7/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9005 - val_loss: 1.0155
Epoch 8/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8490 - val_loss: 1.0045
Epoch 9/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8114 - val_loss: 0.9942
Epoch 10/50
938/938 [==============================] - 18s 19ms/step - loss: 0.775

## 모델 구현하기

### 인코더

In [23]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 512)         2967040   
_________________________________________________________________
masking (Masking)            (None, None, 512)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 512), (None, 512) 2099200   
Total params: 5,066,240
Trainable params: 5,066,240
Non-trainable params: 0
_________________________________________________________________


### 디코더

In [24]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [25]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [26]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 512)    4248064     decoder_input[0][0]              
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
____________________________________________________________________________________________

In [27]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [28]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [29]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

## 모델 평가하기

In [30]:
for seq_index in [1,201,501,1004,2015]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: the food looks delicious . 
정답 문장: les plats ont l air d licieux . 
번역기가 번역한 문장:  le sais le le le 
-----------------------------------
입력 문장: i m allergic to gluten . 
정답 문장: je suis allergique au gluten . 
번역기가 번역한 문장:  je suis . de les au
-----------------------------------
입력 문장: that s just what he needs . 
정답 문장: c est pr cis ment ce qu il lui faut . 
번역기가 번역한 문장:  c n pr ce ce just
-----------------------------------
입력 문장: i hope you re happy , too . 
정답 문장: j esp re que vous tes galement heureuse . 
번역기가 번역한 문장:  j esp entendu vou
-----------------------------------
입력 문장: three of these are tom s . 
정답 문장: il y en a trois tom . 
번역기가 번역한 문장:  il y y ce d d . 


## 마무리

입력 문장: i want to ask you something .   
정답 문장: je veux vous demander quelque chose .   
번역기가 번역한 문장:  je veux te vous quelqu  

입력 문장: stop playing hard to get .   
정답 문장: cessez de faire ceux qui ne sont pas int ress s !  <br>
번역기가 번역한 문장:  arr de de faire fair  

입력 문장: you never have any money .   
정답 문장: tu ne disposes jamais d aucun argent .   
번역기가 번역한 문장:  tu n jamais d d d d   

입력 문장: tom slipped and nearly fell .   
정답 문장: tom <unk> et <unk> tomber .   
번역기가 번역한 문장:  tom a les et en en   

입력 문장: tom kept the window closed .   
정답 문장: tom garda la fen tre ferm e .   
번역기가 번역한 문장:  tom tom la la la la l  

번역기가 번역한 문장이 완벽하지는 않지만, 어느정도 비슷한 결과를 보이고 있습니다.  
학습 데이터가 더 많거나, 전처리 부분에 신경을 많이 쓴다면 더 좋은 결과를 얻을 것이라고 생각 됩니다.  